In [ ]:
from comet_ml import Experiment

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from random import choice
import urllib.request  # the lib that handles the url stuff
import time


url = "https://raw.githubusercontent.com/gracexwho/drug-drug-interactions/master/ChCh-Miner_durgbank-chem-chem.tsv"
url_data = urllib.request.urlopen(url) 

G = nx.read_edgelist(url_data)

print(G.number_of_nodes())
print(G.number_of_edges())


# Create an experiment
experiment = Experiment(api_key="yeThLw8MLFuaMF3cVW1b9IsIt",
                        project_name="general", workspace="gracexwho")

# Report any information you need by:

################# CONTROL ##################

hyper_params = {"learning_rate": 0.1, "epochs": 20}
experiment.log_parameters(hyper_params)

num_train = 200
num_val = 50

################# CONTROL ##################

In [12]:
def Jaccard(T):
    JC = {}
    listNodes = list(T.nodes())
    for u in listNodes:
        for v in listNodes:
            common = len(list(nx.common_neighbors(T, u, v)))
            union = T.degree[u] + T.degree[v] - common
            if union == 0:
                JC[(u,v)] = 0
            else:
                JC[(u,v)] = (common/union)
    
    return JC



def Common_Neighbours(T):
    CN = {}
    listNodes = list(T.nodes())
    for u in listNodes:
        for v in listNodes:
            CN[(u,v)] = len(list(nx.common_neighbors(T, u, v)))
    return CN


def Adamic_Adar(T):
    AA = {}
    listNodes = list(T.nodes())
    for u in listNodes:
        for v in listNodes:
            common = nx.common_neighbors(T, u, v)
            AA[(u,v)] = sum(1/np.log(G.degree[common]))
    return AA


Pairs = [(id1, id2) …]

-Define ID map (pre-processing step)
-Before training loop, do

Pairs = (get_id(pair[0]) , get_id(pair[1] for pair in pairs)

Id_map = {}
Def get_id(id_string):
	If id_string is in id_map:
		Return id_map(id_string)
	Else:
		Id = len(id_map)
		id_map(id_string) = Id
		Return Id


For epoch in range(num_epochs):
Shuffle(pairs)
Index = 0
Batch_size = 64
While index+batch_size < length(pairs):
	Batch = pairs[index:index+batch_size]      // index[min(index+batch_size, len(pairs))
	Index += batch_size

Def process_batch(batch):
	Left_ids = LongTensor([pair[0] for pair in batch])
	Right_ids = LongTensor([pair[1] for pair in batch])
	Neg_ids = LongTensor([np.randint(0, maxnodeid) for _ in batch)

	Left_embeds = embedding(left_ids)
	 tensor batch size x embedding dimension
	Right_embeds =
	Neg_embeds =...

	Pos_score = left_embed x right_embed then summed across axis=0
	Neg_score = left_embed x neg_embed

	Loss = get_loss(pos_score, neg_score)

-one-hot encoding
-random walk permutation
	-center node and then do pairing two left and two right


In [13]:
# Get training set validation set

# what if you just test on edges: instead of training on all node x node combos, just train on edges
# for u in training_nodes, for v in list(G.adj[u])


start_time = time.time()


graph_nodes = list(G.nodes())
training_nodes = torch.zeros([1, num_train], dtype=torch.float)
training_nodes = training_nodes
validation_nodes = torch.zeros([1, num_val], dtype=torch.float)
node_dict = {}
# encode Nodes as numbers

index = 0
for node in list(G.nodes()):
    node_dict[index] = node
    index += 1

    
for i in range(num_train):
    training_nodes[0][i] = choice(range(len(node_dict)))
    if i < num_val:
        validation_nodes[0][i] = choice(range(len(node_dict)))

y_true = []
for x in validation_nodes[0]:
    node_x = node_dict[x.item()]
    for y in validation_nodes[0]:
        node_y = node_dict[y.item()]
        if (node_x, node_y) in list(G.edges()):
            y_true.append(1)
        else:
            y_true.append(0)
        

print(training_nodes.shape)
print(validation_nodes.shape)


similarity = Jaccard(G)

print("--- %s minutes ---" % ((time.time() - start_time)//60))

# 11 min for 500 nodes in training_nodes, 100 in val
# 8 min for 200 nodes in training, 50 in val

torch.Size([1, 200])
torch.Size([1, 50])
--- 8.0 minutes ---


In [14]:
# Now creating the mapping
# map nodes -> R^d
# decoder in training
# lesson 7: Encoders-Decoders

class Encoder(nn.Module):
  # should return VECTORS for each node
    def __init__(self, num_train):
        super(Encoder, self).__init__()
        
        #self.dropout = nn.Dropout(p=0.2)
        # one layer, return embeds(node_ids) which is a long tensor
        #learnrate might be too big if doesn't decrease
        self.embed = nn.Embedding(num_train, 256)

    
    def forward(self, x):
        x = self.embed(x)
        return x

# embeds.weight = embeds.weight/np.sqrt(mbed_dim)
  # Loss function can't be a float, it should be a tensor
  # and also DON'T unwrap a tensor at any point, that gets rid of grad
  # Keep it in tensor operators: maybe change node_dict into a tensor

    
def LossFunction(nodes, similarity):
  # decoder represents ALREADY DECODED results
  # similarity represents evaluated similarities
  # decode is a tuple (u,v) of integers
    u, v = nodes
    a = model(torch.cuda.LongTensor([[u]]))            #input to a model on CUDA should be CUDA tensors!
    b = model(torch.cuda.LongTensor([[v]]))
  
    decoded = Decoder(a,b)
    sim = torch.tensor(similarity[(node_dict[u.item()], node_dict[v.item()])])
    loss = decoded - sim       ##
    loss = torch.pow(loss, 2)
    return loss
  
# calculate inner product
def Decoder(v1, v2):
    v1 = torch.squeeze(v1)
    v2 = torch.squeeze(v2)
    return torch.dot(v1, v2)
        

In [16]:
## Train
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import zero_one_loss


start_time = time.time()

model = Encoder(G.number_of_nodes())
model.embed.weight.data = (model.embed.weight.data/np.sqrt(256))
model.cuda()

optimizer = optim.SGD(model.parameters(), lr=hyper_params['learning_rate'])

epochs = hyper_params['epochs']

 # have to somehow get vectors representing each node
 # pass in a SINGLE NODE to encoder and get back a vector
 # Comet.ml

encoded = {}
difference = float("inf")




for e in range(epochs):
    train_loss = 0
    running_loss = 0
    
    model.train()
    for u in training_nodes[0]:
        for v in training_nodes[0]:
            optimizer.zero_grad()
            train_loss = LossFunction((u, v), similarity)
            running_loss += train_loss.item()
            train_loss.backward()        #retain_graph=True
            optimizer.step()
            
            

    print(f"Training loss: {running_loss}")
    
    
    model.eval()
    y_score = []
    index = []
    
    for x in validation_nodes[0]:
        for y in validation_nodes[0]:
            a = model(torch.cuda.LongTensor([[x]])) 
            b = model(torch.cuda.LongTensor([[y]]))  
            result = Decoder(a, b)
            result = result.cpu()
            result = result.detach().numpy()
            y_score.append(result)
            index.append(similarity[(node_dict[u.item()], node_dict[v.item()])])
            
    new_auc = roc_auc_score(y_true, y_score)
    print("The AUC score is: ", new_auc)
    index_auc = roc_auc_score(y_true, index)
    print("AUC for JC index is: ", index_auc)
    
    if (abs(new_auc - index_auc) < difference):
        print("Saving model...")
        torch.save(model.state_dict(), 'encoder-decoder.pth')
    difference = abs(new_auc - index_auc)
    
    

print(model.embed.weight.data)

print("--- %s minutes ---" % ((time.time() - start_time)//60))

#  for 500 training, 100 val
# for 200 training, 50 val --- 33.0 minutes ---
#Training loss: 0.01853493543912457
#The AUC score is:  0.4338612244897959
#AUC for JC index is:  0.5
    
    

Training loss: 189.76642837799488
The AUC score is:  0.4356897959183673
AUC for JC index is:  0.5
Saving model...
Training loss: 25.19827439197786
The AUC score is:  0.43366530612244897
AUC for JC index is:  0.5
Training loss: 8.696022829730163
The AUC score is:  0.43343673469387756
AUC for JC index is:  0.5
Training loss: 3.4490601763001845
The AUC score is:  0.43407346938775504
AUC for JC index is:  0.5
Training loss: 1.5137561598994025
The AUC score is:  0.4341877551020409
AUC for JC index is:  0.5
Training loss: 0.7385860222791126
The AUC score is:  0.43405714285714286
AUC for JC index is:  0.5
Training loss: 0.40077482316904467
The AUC score is:  0.4339591836734694
AUC for JC index is:  0.5
Training loss: 0.23992620998484948
The AUC score is:  0.4339265306122449
AUC for JC index is:  0.5
Training loss: 0.15649955039700528
The AUC score is:  0.43386122448979597
AUC for JC index is:  0.5
Training loss: 0.10963670454440388
The AUC score is:  0.43397551020408165
AUC for JC index is:  

### Result 6-3-2019

#### Trends
##### The AUC score doesn't really change, and it approaches the JC score

Training loss: 206.67951894763414
The AUC score is:  0.47640592225131295
AUC for JC index is:  0.5
Saving model...
Training loss: 29.62321179928426
The AUC score is:  0.47848820296112565
AUC for JC index is:  0.5
Saving model...
Training loss: 10.369048358812345
The AUC score is:  0.47847799570274424
AUC for JC index is:  0.5
Saving model...
Training loss: 4.126018026217311
The AUC score is:  0.4784882029611256
AUC for JC index is:  0.5
Saving model...
Training loss: 1.8465247948907366
The AUC score is:  0.47844737392759984
AUC for JC index is:  0.5
Saving model...
Training loss: 0.9538654205311691
The AUC score is:  0.4785800682865586
AUC for JC index is:  0.5
Saving model...
Training loss: 0.5615920932513642
The AUC score is:  0.4786515190952285
AUC for JC index is:  0.5
Saving model...
Training loss: 0.3670247585797502
The AUC score is:  0.47865151909522863
AUC for JC index is:  0.5
Saving model...
Training loss: 0.2596153598021152
The AUC score is:  0.47862089732008434
AUC for JC index is:  0.5
Saving model...
Training loss: 0.1948714980473346
The AUC score is:  0.4785494465114143
AUC for JC index is:  0.5
Saving model...
tensor([[ 0.0036,  0.0317, -0.0416,  ..., -0.1481,  0.0023, -0.0238],
        [ 0.0305, -0.0064, -0.0720,  ...,  0.0456, -0.0369, -0.1447],
        [ 0.0626, -0.0218,  0.0999,  ..., -0.0788,  0.0204, -0.0226],
        ...,
        [ 0.0026,  0.0091, -0.0706,  ...,  0.0365, -0.0633,  0.1414],
        [ 0.0566,  0.0866,  0.0814,  ...,  0.0723,  0.0770,  0.0155],
        [-0.0343, -0.0056, -0.0170,  ...,  0.0677, -0.0670,  0.1061]],
       device='cuda:0')
--- 20.0 minutes ---